# GrimAge2PackYrs

## Index
1. [Instantiate model class](#Instantiate-model-class)
2. [Define clock metadata](#Define-clock-metadata)
3. [Download clock dependencies](#Download-clock-dependencies)
5. [Load features](#Load-features)
6. [Load weights into base model](#Load-weights-into-base-model)
7. [Load reference values](#Load-reference-values)
8. [Load preprocess and postprocess objects](#Load-preprocess-and-postprocess-objects)
10. [Check all clock parameters](#Check-all-clock-parameters)
10. [Basic test](#Basic-test)
11. [Save torch model](#Save-torch-model)
12. [Clear directory](#Clear-directory)

Let's first import some packages:

In [18]:
import os
import inspect
import shutil
import json
import torch
import pandas as pd
import pyaging as pya
import numpy as np

## Instantiate model class

In [19]:
def print_entire_class(cls):
    source = inspect.getsource(cls)
    print(source)

print_entire_class(pya.models.GrimAge2PackYrs)

class GrimAge2PackYrs(pyagingModel):
    def __init__(self):
        super().__init__()

    def preprocess(self, x):
        return x

    def postprocess(self, x):
        return x



In [20]:
model = pya.models.GrimAge2PackYrs()

## Define clock metadata

In [21]:
model.metadata["clock_name"] = 'grimage2packyrs'
model.metadata["data_type"] = 'methylation'
model.metadata["species"] = 'Homo sapiens'
model.metadata["year"] = 2022
model.metadata["approved_by_author"] = '⌛'
model.metadata["citation"] = "Lu, Ake T., et al. \"DNA methylation GrimAge version 2.\" Aging (Albany NY) 14.23 (2022): 9484."
model.metadata["doi"] = "https://doi.org/10.18632/aging.204434"
model.metadata["research_only"] = True
model.metadata["notes"] = None

## Download clock dependencies

In [22]:
logger = pya.logger.Logger()
urls = [
    "https://pyaging.s3.amazonaws.com/supporting_files/grimage2_subcomponents.csv",
    "https://pyaging.s3.amazonaws.com/supporting_files/grimage2.csv",
    "https://pyaging.s3.amazonaws.com/supporting_files/datMiniAnnotation3_Gold.csv",
]
dir = "."
for url in urls:
    pya.utils.download(url, dir, logger, indent_level=1)

|-----------> Downloading data to ./grimage2_subcomponents.csv
|-----------> in progress: 100.0000%
|-----------> Downloading data to ./grimage2.csv
|-----------> in progress: 100.0000%%
|-----------> Downloading data to ./datMiniAnnotation3_Gold.csv
|-----------> in progress: 100.0000%


## Load features

#### From CSV

In [23]:
df = pd.read_csv('grimage2_subcomponents.csv', index_col=0)
df['Y.pred'].unique()

array(['DNAmGDF_15', 'DNAmB2M', 'DNAmCystatin_C', 'DNAmTIMP_1', 'DNAmadm',
       'DNAmpai_1', 'DNAmleptin', 'DNAmPACKYRS', 'DNAmlog.CRP',
       'DNAmlog.A1C'], dtype=object)

In [24]:
df = df[df['Y.pred'] == 'DNAmPACKYRS']
df['feature'] = df['var']
df['coefficient'] = df['beta']
model.features = ['age'] + df['feature'][2:].tolist()

In [25]:
df.head()

,Y.pred,var,beta,feature,coefficient
1622,DNAmPACKYRS,Intercept,-31.997022,Intercept,-31.997022
1623,DNAmPACKYRS,Age,0.142144,Age,0.142144
1624,DNAmPACKYRS,cg20462449,14.697949,cg20462449,14.697949
1625,DNAmPACKYRS,cg19802390,0.459989,cg19802390,0.459989
1626,DNAmPACKYRS,cg14084907,0.382296,cg14084907,0.382296


## Load weights into base model

#### Linear model

In [26]:
weights = torch.tensor(df['coefficient'][1:].tolist()).unsqueeze(0)
intercept = torch.tensor([df['coefficient'].iloc[0]])

#### Linear model

In [27]:
base_model = pya.models.LinearModel(input_dim=len(model.features))

base_model.linear.weight.data = weights.float()
base_model.linear.bias.data = intercept.float()

model.base_model = base_model

## Load reference values

In [28]:
reference_df = pd.read_csv('datMiniAnnotation3_Gold.csv', index_col=0)
model.reference_values = [65] + reference_df.loc[model.features[1:]]['gold'].tolist()

## Load preprocess and postprocess objects

In [29]:
model.preprocess_name = None
model.preprocess_dependencies = None

In [30]:
model.postprocess_name = None
model.postprocess_dependencies = None

## Check all clock parameters

In [31]:
pya.utils.print_model_details(model)


%==================================== Model Details ====================================%
Model Attributes:

training: True
metadata: {'approved_by_author': '⌛',
 'citation': 'Lu, Ake T., et al. "DNA methylation GrimAge version 2." Aging '
             '(Albany NY) 14.23 (2022): 9484.',
 'clock_name': 'grimage2packyrs',
 'data_type': 'methylation',
 'doi': 'https://doi.org/10.18632/aging.204434',
 'notes': None,
 'research_only': True,
 'species': 'Homo sapiens',
 'version': None,
 'year': 2022}
reference_values: [65, 0.936110502255403, 0.46754473577805, 0.101838529204782, 0.668699052138661, 0.408180706152194, 0.033798963669035, 0.044827788203083, 0.91516014793103, 0.855444378968262, 0.189367366387564, 0.949642624900366, 0.430182534715076, 0.550120185761743, 0.382373523551351, 0.151011071218822, 0.750308469962453, 0.0481653215294158, 0.0623779128286142, 0.795309058162839, 0.911175830780723, 0.0419140679935733, 0.0388065967727853, 0.854286856105627, 0.514045648234368, 0.935109546405548

## Basic test

In [32]:
torch.manual_seed(42)
input = torch.randn(10, len(model.features), dtype=float).double()
model.eval()
model.to(float)
pred = model(input)
pred

tensor([[  17.0963],
        [ 118.3757],
        [-151.4175],
        [-116.8670],
        [ 181.9866],
        [  -9.3648],
        [  91.3053],
        [-235.3991],
        [ -64.6384],
        [  68.6570]], dtype=torch.float64, grad_fn=<AddmmBackward0>)

## Save torch model

In [33]:
torch.save(model, f"../weights/{model.metadata['clock_name']}.pt")

## Clear directory
<a id="10"></a>

In [34]:
# Function to remove a folder and all its contents
def remove_folder(path):
    try:
        shutil.rmtree(path)
        print(f"Deleted folder: {path}")
    except Exception as e:
        print(f"Error deleting folder {path}: {e}")

# Get a list of all files and folders in the current directory
all_items = os.listdir('.')

# Loop through the items
for item in all_items:
    # Check if it's a file and does not end with .ipynb
    if os.path.isfile(item) and not item.endswith('.ipynb'):
        os.remove(item)
        print(f"Deleted file: {item}")
    # Check if it's a folder
    elif os.path.isdir(item):
        remove_folder(item)

Deleted file: grimage2_subcomponents.csv
Deleted file: datMiniAnnotation3_Gold.csv
Deleted file: grimage2.csv
